In [1]:
import os
import json

import pandas as pd
import numpy as np

import dash
# from dash import Dash
# import jupyter_dash as dash
from jupyter_dash import JupyterDash as Dash
import dash_bootstrap_components as dbc
from dash import Input, Output, html, dcc

import plotly
import plotly.express as px
import plotly.figure_factory as ff

In [2]:
class DataModel:
    # Constants
    output_options = ['Mean issue', 'Volatility issue', 'Input data issue', 'Model issue', 'Others']
    status_options = ['Not Validated', 'Approved', 'Denied']
    
    # Sidebar
    dates = []
    current_date = None
    validations = []
    current_validation = None
    pages = []
    path_to_pages = dict()
    
    # Content
    outputs = dict()
    
    @classmethod
    def get_category_outputs(cls, category):
        return [cls.outputs[key] for key in cls.outputs if key.split(';')[0] == category]
    
    class Output:
        def __init__(self, category, name, description, data, options=None, comments=None, status=None):
            self.id = f'{category};{name}'
            self.category = category
            self.name = name
            self.description = description
            self.data = plotly.io.read_json(f'data/{DataModel.current_date}/{data}')
            self.options = [] if options is None else options
            self.comments = comments
            self.status = DataModel.status_options[0] if status is None else status

        def set_options(self, options):
            self.options = options

        def set_comments(self, comments):
            self.comments = comments

        def set_status(self, status):
            self.status = status

        def update(self, options, comments, status):
            self.set_options(options)
            self.set_comments(comments)
            self.set_status(status)
    
    @classmethod
    def refresh_dates(cls):
        cls.dates = sorted([name for name in os.listdir('data/') if os.path.isdir(f'data/{name}')], reverse=True)
        cls.current_date = cls.dates[0]
    
    @classmethod
    def refresh_validations(cls, date):
        cls.validations = [name[:-4] for name in os.listdir(f'data/{date}/') if name.split('.')[-1] == 'val']
        cls.current_validation = None
    
    @classmethod
    def update_output(cls, key, output):
        cls.outputs[key] = output
    
    @classmethod
    def _get_validation_data(cls, date, validation=None):
        if validation is None:
            dfs = [pd.read_csv(f'data/{date}/{name}') for name in os.listdir(f'data/{date}/') if name.split('.')[-1] == 'meta']
            df = pd.concat(dfs, axis=0)
            df = df.reindex(columns=['category', 'name', 'description', 'data', 'options', 'comments', 'status'])
        else:
            df = pd.read_csv(f'data/{date}/{validation}.val')
        df.index = df[['category', 'name']].agg(';'.join, axis=1)
        return df
    
    @classmethod
    def refresh_outputs_and_pages(cls):
        df = cls._get_validation_data(cls.current_date, cls.current_validation)
        for key in df.index:
            cls.update_output(key, cls.Output(**{k: None if v is np.nan else v for k, v in df.loc[key].to_dict().items()}))
        cls.pages = list(df['category'].unique())
        cls.path_to_pages = {p.replace(' ', '_'): p for p in cls.pages}
        
    @classmethod
    def init(cls):
        cls.refresh_dates()
        cls.refresh_validations(cls.current_date)
        cls.refresh_outputs_and_pages()

In [3]:
DataModel.init()

In [4]:
class OutputComponent:
    @staticmethod
    def get_component(output: Output):
        component = html.Div(children=[
            html.H3(output.name, className="display-6"),
            dcc.Graph(figure=output.data),
            dcc.Markdown(output.description),
            
            dbc.FormText("Issue(s):"),
            dcc.Dropdown(DataModel.output_options, output.options, id=f'{output.id};options', multi=True),
            
            dbc.FormText("Comments:"),
            dbc.Input(placeholder="Validation Comments", id=f'{output.id};comments'),
            
            dbc.FormText("Validation Status:"),
            dcc.Dropdown(DataModel.status_options, output.status, id=f'{output.id};status'),
            
            dbc.Button("Primary", color="primary", className="me-1", id=f'{output.id};button'),
            html.Hr()
        ])
    
        @app.callback(Output('Placeholder', 'children'), 
                      Input(f'{output.id};button', 'n_clicks'))
                      # *[Input(f'{output.id};{c}', 'value') for c in ['options', 'comments', 'status']])
        def save_updates(n_clicks):
            if n_clicks is not None:
                return "Something works"
            # return "Entered"
            # ctx = dash.callback_context
            # if ctx.triggered:
            #     id_ = ';'.join(ctx.triggered[0]['prop_id'].split(';')[:-1])
            #     DataModel.outputs[id_].update(options, comments, status)
            #     return f"ctx was triggered"
            # else:
            #     return "ctx not triggered"
            
        return component

In [5]:
class SideBar:
    SIDEBAR_STYLE = {
        "position": "fixed",
        "top": 0,
        "left": 0,
        "bottom": 0,
        "width": "16rem",
        "padding": "2rem 1rem",
        "background-color": "#f8f9fa",
    }
    
    def __init__(self):
        date_validation_selection = [                
            html.H4("Options"),
            dbc.FormText("Date:"),
            dcc.Dropdown(DataModel.dates, DataModel.current_date, id='SideBar;dates'),
            dbc.FormText("Validation:"),
            dcc.Dropdown(DataModel.validations, DataModel.current_validation, id='SideBar;validations'),         
            dbc.FormText("Submit:"),
            html.Div(dbc.Button("Submit", color="primary"), className="d-grid gap-2")
        ]
        
        pages = [
            html.H4("Pages"),
            dbc.Nav(
                [
                    dbc.NavLink("Home", href="/", active="exact"),
                    *[dbc.NavLink(v, href=f"/{k}", active="exact") for k, v in DataModel.path_to_pages.items()]
                ],
                vertical=True,
                pills=True,
            )
        ]
        
        self.sidebar = html.Div(
            [   
                *date_validation_selection,
                html.Hr(),
                *pages
            ],
            style=SideBar.SIDEBAR_STYLE,
        )
        
        @app.callback(Output('Placeholder', 'children'), 
                      Input('SideBar;dates', 'value'), 
                      Input('SideBar;validations', 'value'))
        def update_date(date, validation):
            DataModel.current_date = date
            DataModel.current_validation = validation
            # return f"Changed Dates to {date}"
    
    def get_sidebar(self):
        return self.sidebar

In [6]:
class MainPage:
    PAGE_STYLE = {
        "margin-left": "16rem",
        "padding": "2rem",
    }
    def __init__(self):
        self.page = html.Div(
            dbc.Container([
                html.Div(id='Placeholder'),
                html.H1("Validation App", className="display-3"),
                html.P("Frontend for oneModel Demo", className="lead"),
                html.Hr(className="my-2"),
                html.Div(id="page-content")
            ]),
            style=MainPage.PAGE_STYLE
        )
        
    def get_page(self):
        return self.page

In [7]:
app = Dash(__name__, external_stylesheets=[dbc.themes.LUX])
sb = SideBar().get_sidebar()
mp = MainPage().get_page()
content = html.Div([dcc.Location(id="url"), sb, mp])

In [8]:
app.layout = content

In [9]:
@app.callback(Output("page-content", "children"), [Input("url", "pathname")])
def render_page_content(pathname):
    if pathname == "/":
        return html.P("This is the content of the home page!")
    
    category = DataModel.path_to_pages[pathname[1:]]
    category_outputs = DataModel.get_category_outputs(category)
    if len(category_outputs) != 0:
        return [OutputComponent.get_component(o) for o in category_outputs]
    
    # If the user tries to reach a different page, return a 404 message
    return html.Div(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )

In [10]:
DataModel.current_date

'20220307'

In [11]:
app.run_server(mode="inline", debug=True)